# Methodology
Risk is calculated by disporportionality in demographics.

## For Normal People:
"Risk" is your chance of being infected or dying while in a demographic compared to the average.

Risk is how much your characteristics affect how likely you are to catch or die of covid.


risk is the ratio of the observed porportion of your demographic in the infected and your demographic in the population
## For nerds:

Risk is calculated using a naive bayes approach, where we take the probability of being affected given being in a demographic.


Risk = (Affected_Demographic/Affected)/(Demographic/Population)

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

A = Affected, B = Demo

$$P(Affected \cap Demo) = \frac{\#Affected Demo}{\#Demo} \\

P(Demo) = \frac{\#Demo}{\#Total Population} \\

P(Affected|Demo) = \frac{P(Affected \cap Demo)}{P(Demo)} \\

\\$$

In [17]:
import pandas as pd

raw_files = ['sex__all_age_groups', 'race_ethnicity__all_age_groups', 'age_group']
index_names = ['Sex', 'Race/Ethnicity', 'Age Group']
cleaned_files = ['sex_data', 'ethnicity_data', 'age_data']

for handle, index, clean in zip(raw_files, index_names, cleaned_files):
  case = pd.read_csv(f'data/cases_by_{handle}.csv', header=0, comment='#')
  death = pd.read_csv(f'data/deaths_by_{handle}.csv', header=0, comment='#').drop('Percent of US population',axis=1)

  # deaths by sex has a 95% confidence interval instead of a count
  if handle == 'sex__all_age_groups':
    death['Count of deaths'] = pd.to_numeric(death['Count of deaths'].apply(lambda x: x.split(' - ')[1]), errors='coerce')

  df = case.merge(death, on=index, how='left')
  df['dc_ratio'] = df['Count of deaths']/df['Count of cases']
  df['Portion of cases'] = df['Count of cases']/df['Count of cases'].sum()*100
  df['Portion of deaths'] = df['Count of deaths']/df['Count of deaths'].sum()*100
  df['Portion of US population'] = df['Percent of US population']
  # Risk is calculated as the Affected
  df['Case Risk'] = df['Portion of cases']/df['Portion of US population']
  df['Death Risk'] = df['Portion of deaths']/df['Portion of US population']

  df = df[[index, 'Case Risk', 'Death Risk', 'Portion of cases', 'Portion of deaths', 'Portion of US population', 'Count of cases', 'Count of deaths', 'dc_ratio']]
  df.to_csv(f'data/{clean}.csv', index=False)



In [18]:
import pandas as pd
cleaned_files = ['sex_data','ethnicity_data','age_data', 'vax_data']

for handle in cleaned_files:
  with open(f'data/{handle}.json','w+') as f:
    df = pd.read_csv(f'data/{handle}.csv', header=0, names=['value', 'cases', 'deaths', 'case porp', 'death porp', 'pop', 'Case count', 'Death count', 'dc_ratio'], comment='#')
    df = df[['value', 'pop', 'cases', 'deaths', 'dc_ratio']]
    df = df.append({'value':'-', 'pop':0, 'cases':0, 'deaths':0, 'dc_ratio':0},ignore_index=True)
    f.write(f'{{ "{handle}": {df.to_json(orient="records")} }}')
    f.close()



In [25]:
import json
from string import ascii_lowercase as alpha
cleaned_files = ['sex_data','ethnicity_data','age_data', 'vax_data']
vals = []

for handle in cleaned_files:
  vals.append(pd.read_csv(f'data/{handle}.csv', header=0, names=['value', 'cases', 'deaths', 'case porp', 'death porp', 'pop', 'Case count', 'Death count', 'dc_ratio'], comment='#')['value'].values)

labels = ['sex', 'ethnicity', 'age','vaccination']

a = 0
fields = open('fields.json', 'w+')
fields.write(f'{{"options" :{{\n')
for i, (val, label) in enumerate(zip(vals,labels)):
  fields.write(f'   "{label}" : [\n')
  for v in val: # if letters run out, double them
    fields.write(f'    {{"key" : "{alpha[a%26]*(1+a//26)}", "text": "{v}", "value": "{v}"}},\n')
    a += 1

  # write last item without trailing comma
  fields.write(f'    {{"key" : "{alpha[a%26]*(1+a//26)}", "text": "-", "value": "-"}}\n')
  a += 1
  
  if i == len(labels)-1:
    fields.write('  ]\n')
  else:
    fields.write('  ],\n')
fields.write('}}')
fields.close()


In [22]:
alpha

'abcdefghijklmnopqrstuvwxyz'